# Map Visuals

This notebook creates interactive map visualization tools with Folium

In [137]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import folium
import branca.colormap as cm
from folium import IFrame
from folium.plugins import MarkerCluster
from folium.plugins import Search

# MarkerCluster()

In [96]:
def embed_map(m,number):
    from IPython.display import IFrame

    m.save(f'index{number}.html')
    return IFrame(f'index{number}.html', width='100%', height='750px')

In [97]:
shapefile = gpd.read_file("tl_2010_25025_tract10/tl_2010_25025_tract10.shp")
print(shapefile)

    STATEFP10 COUNTYFP10 TRACTCE10      GEOID10   NAME10  \
0          25        025    010405  25025010405   104.05   
1          25        025    010404  25025010404   104.04   
2          25        025    010801  25025010801   108.01   
3          25        025    010702  25025010702   107.02   
4          25        025    010204  25025010204   102.04   
5          25        025    010802  25025010802   108.02   
6          25        025    010104  25025010104   101.04   
7          25        025    000703  25025000703     7.03   
8          25        025    000504  25025000504     5.04   
9          25        025    000704  25025000704     7.04   
10         25        025    010103  25025010103   101.03   
11         25        025    000803  25025000803     8.03   
12         25        025    980300  25025980300     9803   
13         25        025    120201  25025120201  1202.01   
14         25        025    120104  25025120104  1201.04   
15         25        025    110607  2502

In [98]:
df_geo = gpd.GeoDataFrame(shapefile)

In [99]:
df_geo['id']=df_geo['STATEFP10']+df_geo['COUNTYFP10']+df_geo['TRACTCE10']

In [100]:
df_geo = df_geo[['geometry','id']]

## Population change

In [101]:
df_data = pd.read_csv('fullcount_final.csv',index_col=0)

In [102]:
df_pop = df_data[['tractid','POP00_POC']]

In [103]:
df_pop['id']=df_pop['tractid'].astype('str')
df_pop.drop(columns=['tractid'],axis=1,inplace=True)
df_tovisualize = df_geo.merge(df_pop, on='id')

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
df_tovisualize['POP00_POC'].quantile([0,0.25,0.5,0.75,1])

0.00    -99.548410
0.25     -3.902751
0.50      2.356902
0.75     10.306031
1.00    132.961310
Name: POP00_POC, dtype: float64

In [173]:
# create map for population changes in Boston
colormap = cm.StepColormap(colors=['#ca0020','#f4a582','#92c5de','#0571b0'],
    index=df_tovisualize['POP00_POC'].quantile([0,0.25,0.5,0.75,1]),
                             vmin = -99.548410 ,vmax = 132.9613,caption='Change in Population 2000-2010'
)

mapobj = folium.Map(location=[42.3601, -71.0589],zoom_start=13,tiles='Stamen Terrain')
style_function = lambda x: {
    'fillColor': colormap(x['properties']['POP00_POC']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.7
}
tractgeo=folium.features.GeoJson(data= df_tovisualize.to_json(),
                        name='Boston Population',
                        style_function=style_function,
                        tooltip=folium.GeoJsonTooltip(fields=['id','POP00_POC'],
                                                     aliases=['Tract','Population change'],
                                                     localize=True)).add_to(mapobj)
tractsearch = Search(
    layer=tractgeo,
    geom_type='Polygon',
    placeholder='Search for a census tract',
    collapsed=False,
    search_label='id'
).add_to(mapobj)

colormap.add_to(mapobj)

In [174]:
embed_map(mapobj,1)

## MEDIAN RENT CHANGE

In [37]:
df_data2 = pd.read_csv('samplecount_final.csv',index_col=0)

In [38]:
df_rent = df_data2[['tractid','MRENT00_POC']]
df_rent['id']=df_data2['tractid'].astype('str')

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
df_rent.drop(columns=['tractid'],axis=1,inplace=True)
df_tovisualize2 = df_geo.merge(df_rent, on='id')

In [176]:
df_tovisualize2['MRENT00_POC'].quantile([0,0.25,0.5,0.75,1])

0.00   -268.498675
0.25     41.169740
0.50     51.933174
0.75     64.153846
1.00    226.734346
Name: MRENT00_POC, dtype: float64

In [187]:
# create map for median rent change in Boston
colormap = cm.StepColormap(colors=['#7b3294','#c2a5cf','#a6dba0','#008837'],
    index=df_tovisualize2['MRENT00_POC'].quantile([0,0.25,0.5,0.75,1]),
                             vmin = -268.498675 ,vmax = 226.734346,caption='Change in Median Rent 2000-2010'
)

mapobj = folium.Map(location=[42.3601, -71.0589],zoom_start=13,tiles='Stamen Terrain')
style_function = lambda x: {
    'fillColor': colormap(x['properties']['MRENT00_POC']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.7
}
tractgeo=folium.features.GeoJson(data= df_tovisualize2.to_json(),
                        name='Boston Rent',
                        style_function=style_function,
                        tooltip=folium.GeoJsonTooltip(fields=['id','MRENT00_POC'],
                                                     aliases=['Tract','Change in Median Rent'],
                                                     localize=True)).add_to(mapobj)
tractsearch = Search(
    layer=tractgeo,
    geom_type='Polygon',
    placeholder='Search for a census tract',
    collapsed=False,
    search_label='id'
).add_to(mapobj)

colormap.add_to(mapobj)

In [188]:
embed_map(mapobj,2)

## MEDIAN HOME VAL

In [62]:
df_home = df_data2[['tractid','MHMVAL00_POC']]
df_home['id']=df_data2['tractid'].astype('str')

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
df_home.drop(columns=['tractid'],axis=1,inplace=True)
df_tovisualize3 = df_geo.merge(df_home, on='id')

In [189]:
df_tovisualize3['MHMVAL00_POC'].quantile([0,0.25,0.5,0.75,1])

0.00   -426.250851
0.25     60.601614
0.50     75.964231
0.75     93.841808
1.00    658.879840
Name: MHMVAL00_POC, dtype: float64

In [190]:
# create map for median home value change in Boston
colormap = cm.StepColormap(colors=['#b3e2cd','#fdcdac','#cbd5e8','#f4cae4'],
    index=df_tovisualize3['MHMVAL00_POC'].quantile([0,0.25,0.5,0.75,1]),
                             vmin = -426.25081 ,vmax = 658.879840,caption='Change in Median Home Value 2000-2010'
)

mapobj = folium.Map(location=[42.3601, -71.0589],zoom_start=13,tiles='Stamen Terrain')
style_function = lambda x: {
    'fillColor': colormap(x['properties']['MHMVAL00_POC']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.7
}
tractgeo=folium.features.GeoJson(data= df_tovisualize3.to_json(),
                        name='Boston Rent',
                        style_function=style_function,
                        tooltip=folium.GeoJsonTooltip(fields=['id','MHMVAL00_POC'],
                                                     aliases=['Tract','Change in Median Home Value'],
                                                     localize=True)).add_to(mapobj)
tractsearch = Search(
    layer=tractgeo,
    geom_type='Polygon',
    placeholder='Search for a census tract',
    collapsed=False,
    search_label='id'
).add_to(mapobj)

colormap.add_to(mapobj)

In [191]:
embed_map(mapobj,3)